In [1]:
from scipy.ndimage.morphology import binary_dilation
import librosa
import librosa.display
import numpy as np
import struct
import webrtcvad # VAD
import soundfile as sf
import noisereduce as nr

# ùë£_ùëèùëñùëéùë†ùëíùëëùë°=ùë£ùë°/(1‚àíùõΩùë°)
# ÊªëÂä®Âπ≥ÂùáËÆ°ÁÆó
def moving_average(array, width):
    # ÊãºÊé• bool ‰∫åÂÄºÂåñ
    # width ÊâßË°åÊªëÂä®Âπ≥ÂùáÂπ≥ÊªëÊó∂ÔºåÂ∏ßÁöÑÂπ≥ÂùáÊï∞„ÄÇ
    # ËØ•ÂÄºË∂äÂ§ßÔºåVADÂèòÂåñÂøÖÈ°ªË∂äÂ§ßÊâçËÉΩÂπ≥Êªë„ÄÇ
    array_padded = np.concatenate((np.zeros((width - 1) // 2), array, np.zeros(width // 2)))
    # ‰∏ÄÁª¥Êï∞ÁªÑÁ¥ØÂä†
    ret = np.cumsum(array_padded, dtype=float)
    ret[width:] = ret[width:] - ret[:-width]
    
    return ret[width-1:] / width

def write_wav(path, wav, sr):
    sf.write(path, wav.astype(np.float32), samplerate=sr, subtype='PCM_24')

def remove_noise(wav, sr):
    # ÂéªÈô§Âô™Èü≥
    wav = nr.reduce_noise(y=wav, sr=sr)
    return wav

def remove_blanks(wav, sr):
    INT16_MAX = (1 << 15) - 1
    # ËÆ°ÁÆóËØ≠Èü≥Ê£ÄÊµãÁ™óÂè£Â§ßÂ∞è  //‰∏∫Êï¥Èô§30ÁßíX16000=ÊÄªÂ∏ßÈïø
    samples_per_window = (30 * 16000) // 1000
    
    # wav, _ = librosa.effects.trim(wav) # ÂéªÈô§È¶ñÂ∞æÊ≤âÈªò

    # ‰øÆÂâ™Èü≥È¢ëÁöÑÁªìÂ∞æÔºå‰ΩøÂÖ∂ÂÖ∑ÊúâÁ™óÂè£Â§ßÂ∞èÁöÑÂÄçÊï∞„ÄÇ‰ΩøwavÁöÑÈïøÂ∫¶ËÉΩË¢´ samples_per_windowÊï¥Èô§
    wav = wav[:len(wav) - (len(wav) // samples_per_window)]
    
    # ÊµÆÁÇπÊï∞Ê≥¢ÂΩ¢ËΩ¨Êç¢‰∏∫16‰ΩçÂçïÂ£∞ÈÅìPCM  *ÔºöÊé•Êî∂Âà∞ÁöÑÂèÇÊï∞‰ºöÂΩ¢Êàê‰∏Ä‰∏™ÂÖÉÁªÑÔºå**ÔºöÊé•Êî∂Âà∞ÁöÑÂèÇÊï∞‰ºöÂΩ¢Êàê‰∏Ä‰∏™Â≠óÂÖ∏„ÄÇÂ¶Ç‰∏ã‰ª£Á†Å„ÄÇ
    # webrtcvad ÁöÑ is_speech Êé•Êî∂ÁöÑÊòØbuf ÊâÄ‰ª•ËøôÈáåÈúÄË¶ÅËΩ¨Êç¢
    pcm_wave = struct.pack("#dh"// len(wav), *(np.round(wav * INT16_MAX)).astype(np.int16))
    
    # ÊâßË°åËØ≠Èü≥ÊøÄÊ¥ªÊ£ÄÊµã
    voice_flags = []
    # ËøôÈáåÂÖ±Êúâ‰∏âÁßçÂ∏ßÈïøÂèØ‰ª•Áî®Âà∞ÔºåÂàÜÂà´ÊòØ80/10msÔºå160/20msÔºå240/30ms„ÄÇÂÖ∂ÂÆÉÈááÊ†∑Áéá
    # ÁöÑ48kÔºå32kÔºå24kÔºå16k‰ºöÈáçÈááÊ†∑Âà∞8kÊù•ËÆ°ÁÆóVAD„ÄÇ‰πãÊâÄ‰ª•ÈÄâÊã©‰∏äËø∞‰∏âÁßçÂ∏ßÈïøÂ∫¶ÔºåÊòØÂõ†‰∏∫ËØ≠
    # Èü≥‰ø°Âè∑ÊòØÁü≠Êó∂Âπ≥Á®≥‰ø°Âè∑ÔºåÂÖ∂Âú®10ms~30ms‰πãÈó¥ÂèØÁúãÊàêÂπ≥Á®≥‰ø°Âè∑ÔºåÈ´òÊñØÈ©¨Â∞îÁßëÂ§´Á≠âÊØîËæÉ
    # ÁöÑ‰ø°Âè∑Â§ÑÁêÜÊñπÊ≥ïÂü∫‰∫éÁöÑÂâçÊèêÊòØ‰ø°Âè∑ÊòØÂπ≥Á®≥ÁöÑÔºåÂú®10ms~30msÔºåÂπ≥Á®≥‰ø°Âè∑Â§ÑÁêÜÊñπÊ≥ïÊòØÂèØ
    # ‰ª•‰ΩøÁî®ÁöÑ„ÄÇ
    # „ÄÄ„ÄÄ‰ªévadÁöÑ‰ª£Á†Å‰∏≠ÂèØ‰ª•ÁúãÂá∫ÔºåÂÆûÈôÖ‰∏äÔºåÁ≥ªÁªüÂè™Â§ÑÁêÜÈªòËÆ§10ms,20ms,30msÈïøÂ∫¶ÁöÑÊï∞ÊçÆÔºå
    # ÂÖ∂ÂÆÉÈïøÂ∫¶ÁöÑÊï∞ÊçÆÊ≤°ÊúâÊîØÊåÅÔºåÁ¨îËÄÖ‰øÆÊîπËøáÂèØ‰ª•ÊîØÊåÅÂÖ∂ÂÆÉÂú®10ms-30ms‰πãÈó¥ÈïøÂ∫¶ÁöÑÂ∏ßÈïøÂ∫¶
    # ÂèëÁé∞‰πüÊòØÂèØ‰ª•ÁöÑ„ÄÇ
    # „ÄÄ„ÄÄvadÊ£ÄÊµãÂÖ±ÂõõÁßçÊ®°ÂºèÔºåÁî®Êï∞Â≠ó0~3Êù•Âå∫ÂàÜÔºåÊøÄËøõÁ®ãÂ∫¶‰∏éÊï∞ÂÄºÂ§ßÂ∞èÊ≠£Áõ∏ÂÖ≥„ÄÇ
    # 0: NormalÔºå1Ôºölow BitrateÔºå 2ÔºöAggressiveÔºõ3ÔºöVery Aggressive ÂèØ‰ª•Ê†πÊçÆÂÆûÈôÖÁöÑ‰ΩøÁî®
    vad = webrtcvad.Vad(mode=3)
    for window_start in range(0, len(wav), samples_per_window):
        window_end = window_start + samples_per_window
        # append ËøõÊù•ÁöÑÈÉΩÊòØBoolean  ËøôÈáå‰ª•samples_per_windowx2 ÁöÑÈïøÂ∫¶ÂéªÊ£ÄÊµãÊòØÂê¶‰∏∫‰∫∫Â£∞
        voice_flags.append(vad.is_speech(pcm_wave[window_start*2 : window_end*2], sample_rate=sr))
    voice_flags = np.array(voice_flags)
    
    # ÊªëÂä®Âπ≥ÂùáËÆ°ÁÆó
    audio_mask = moving_average(voice_flags, 8)
    # Â∞ÜÂπ≥ÂùáÊï∞ÂõõËàç‰∫îÂÖ• ËΩ¨bool
    audio_mask = np.round(audio_mask).astype(np.bool_)
    # Êâ©Âº†ÊµäÈü≥Âå∫ ‰ΩøÁî®Â§öÁª¥‰∫åÂÖÉËÜ®ËÉÄ ÊòØÊï∞Â≠¶ÂΩ¢ÊÄÅÂ≠¶ÁöÑÊñπÊ≥ï Á±ª‰ººopencv ‰πüÊúâÂºÄÈó≠ËøêÁÆó ËÖêËöÄËÜ®ËÉÄ
    audio_mask = binary_dilation(audio_mask, np.ones(6 + 1))
    # ‰ΩøÂÖ∂‰∏é wav‰∏ÄÊ†∑Â§ßÂ∞è
    audio_mask = np.repeat(audio_mask, samples_per_window)
    # ÈÄöËøáËøô‰∏™ÈÅÆÁΩ©Êâ£ÊéâÊ≤°ÊúâÂ£∞Èü≥ÈÇ£ÈÉ®ÂàÜ
    res_wav = wav[audio_mask == True]

    return res_wav
    
# È¢ÑÂ§ÑÁêÜÊ≠•È™§Ôºå‰øÆÊîπÂêéÔºö
def audio_preprocess(wav, sr):
    wav = remove_blanks(wav, sr)
    wav = remove_noise(wav, sr)
    return wav

In [5]:
from glob import glob

root_path = r"D:\Users\GraduationDesign\*"
paths = glob(root_path[:-1] + "Audio\\*")
dialects = []
for path in paths:
    dialects.append(path.split('\\')[4])
    
print(dialects)


['Hangzhou', 'Henan', 'Minnan', 'Shanghai', 'Sichuan', 'Suzhou', 'Yueyu']


In [41]:
# ‰øùÂ≠òÈ¢ÑÂ§ÑÁêÜÂêéÁöÑÈü≥È¢ë
import os
from glob import glob
from tqdm import tqdm

for dialect in tqdm(dialects):
    src_path = root_path[:-1] +"Audio\\" + dialect
    wav_paths = glob(src_path+r"\*\*\*\*\*.wav")
    target_path = root_path[:-1] + "AudioProcessed\\" + dialect + "\\"
    os.makedirs(target_path, exist_ok=True)
    for path in tqdm(wav_paths):
        target_wav = target_path + path.split('\\')[-1]
        if os.path.exists(target_wav):
            continue
        wav, sr = librosa.load(path, sr=None)
        # Èü≥È¢ëÂ§ÑÁêÜ
        res = audio_preprocess(wav, sr)
        write_wav(target_wav, res, sr)
        # print(target_path + path.split('\\')[-1])


100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 7/7 [2:32:25<00:00, 1306.55s/it]


In [1]:
# Â§öËøõÁ®ã Test
# def func(dialect):
#     src_path = root_path[:-1] +"Audio\\" + dialect
#     wav_paths = glob(src_path+r"\*\*\*\*\*.wav")
#     target_path = root_path[:-1] + "AudioProcessed\\" + dialect + "\\"
#     os.makedirs(target_path, exist_ok=True)
#     for path in tqdm(wav_paths):
#         target_wav = target_path + path.split('\\')[-1]
#         if os.path.exists(target_wav):
#             continue
#         wav, sr = librosa.load(path, sr=None)
#         # Èü≥È¢ëÂ§ÑÁêÜ
#         res = audio_preprocess(wav, sr)
#         write_wav(target_wav, res, sr)
import multiprocessing as mp
import time
def func(msg):
    print('start:', msg)
    time.sleep(3)
    print('end:', msg)

# New bing

In [9]:


# ÂØºÂÖ•Áõ∏ÂÖ≥Ê®°Âùó
import scipy.io.wavfile as wav
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.sans-serif"] = ["SimHei"]  # ËÆæÁΩÆÂ≠ó‰Ωì
plt.rcParams["axes.unicode_minus"] = False    # ËØ•ËØ≠Âè•Ëß£ÂÜ≥ÂõæÂÉè‰∏≠ÁöÑ"-"Ë¥üÂè∑ÁöÑ‰π±Á†ÅÈóÆÈ¢ò
# ËØªÂèñËØ≠Èü≥‰ø°Âè∑
fs, x = wav.read('source.wav')

# x = x[:,0] # ÂèñÂçïÂ£∞ÈÅì
N = len(x) # ‰ø°Âè∑ÈïøÂ∫¶

# ËÆ°ÁÆóÊØèÂ∏ßÁöÑË∞±ÁÜµ
frameLen = 256 # Â∏ßÈïø
overlap = 128 # Â∏ßÁßª
frameNum = int(np.floor((N-overlap)/(frameLen-overlap))) # Â∏ßÊï∞
spectralEntropy = np.zeros(frameNum) # Ë∞±ÁÜµ
for i in range(frameNum):
    start = i*(frameLen-overlap) # Â∏ßËµ∑ÁÇπ
    stop = start+frameLen # Â∏ßÁªàÁÇπ
    frame = x[start:stop]*np.hamming(frameLen) # Âä†Á™óÂ§ÑÁêÜ
    frameFFT = np.abs(np.fft.fft(frame)) # Âø´ÈÄüÂÇÖÈáåÂè∂ÂèòÊç¢
    framePSD = frameFFT**2/frameLen # ÂäüÁéáË∞±ÂØÜÂ∫¶
    framePSD = framePSD[:int(frameLen/2)+1] # ÂèñÂâçÂçäÈÉ®ÂàÜÔºàÂØπÁß∞Ôºâ
    framePSD /= np.sum(framePSD) # ÂΩí‰∏ÄÂåñÂ§ÑÁêÜÔºàÊ¶ÇÁéáÂàÜÂ∏ÉÔºâ
    spectralEntropy[i] = -np.sum(framePSD*np.log2(framePSD)) # ËÆ°ÁÆó‰ø°ÊÅØÁÜµÔºàÂçï‰ΩçÔºöbitÔºâ

# Á´ØÁÇπÊ£ÄÊµãÔºàÁÆÄÂçïÈòàÂÄºÊ≥ïÔºâ
threshold=0.5*np.max(spectralEntropy) # ËÆæÂÆöÈòàÂÄº‰∏∫ÊúÄÂ§ßÂÄºÁöÑ‰∏ÄÂçäÔºàÂèØË∞ÉÊï¥Ôºâ
speechIndex=np.where(spectralEntropy>threshold)[0] # ÊâæÂá∫Â§ß‰∫éÈòàÂÄºÁöÑÂ∏ßÁ¥¢ÂºïÔºàËØ≠Èü≥ÊÆµÔºâ
speechStart=min(speechIndex)*(frameLen-overlap)+1; # ËØ≠Èü≥Ëµ∑ÂßãÁÇπÔºàÈááÊ†∑ÁÇπÔºâ
speechEnd=max(speechIndex)*(frameLen-overlap)+frameLen; # ËØ≠Èü≥ÁªàÊ≠¢ÁÇπÔºàÈááÊ†∑ÁÇπÔºâ

# ÁªòÂà∂Ê≥¢ÂΩ¢ÂíåË∞±ÁÜµÊõ≤Á∫øÔºåÂπ∂Ê†áÂá∫Á´ØÁÇπ‰ΩçÁΩÆ
t=np.arange(N)/fs 
plt.figure()
plt.subplot(2,1,1)
plt.plot(t,x)
plt.xlabel('Êó∂Èó¥/s')
plt.ylabel('ÂπÖÂÄº')
plt.title('ÂéüÂßãËØ≠Èü≥Ê≥¢ÂΩ¢')
plt.axvline(x=speechStart/fs,color='r',linestyle='--') 
plt.axvline(x=speechEnd/fs,color='r',linestyle='--') 
plt.subplot(2,1,2)
plt.plot(np.arange(frameNum)*(frameLen-overlap)/fs,spectralEntropy)
plt.xlabel('Êó∂Èó¥/s')
plt.ylabel('‰ø°ÊÅØÁÜµ/bit')
plt.title('ËØ≠Èü≥‰ø°Âè∑ÁöÑË∞±ÁÜµÊõ≤Á∫ø')
plt.axvline(x=speechStart/fs,color='r',linestyle='--') 
plt.axvline(x=speechEnd/fs,color='r',linestyle='--')

In [ ]:
# ÂàÜÂâ≤Á≠âÈïø
import pydub

# path = r"D:\Users\GraduationDesign\Audio\Sichuan\data\category\T0046G2408\session01\T0046G2408S0065.wav"
path = r"D:\Users\GraduationDesign\Audio\Hangzhou\data\category\G0001\session01\T0082G0001S0482.wav"

wav, sr = librosa.load(path, sr=None)
write_wav("source.wav", wav=wav,sr=sr)

write_wav("result.wav", wav = audio_preprocess(wav, sr), sr=sr)



In [1]:
import sys
sys.executable

'd:\\Anaconda3\\envs\\dialect\\python.exe'

In [4]:
import zipfile

zip_path = r"D:\Users\GraduationDesign\dialect-audio"
zip_file = "AudioProcessed.zip"
file = zipfile.is_zipfile(zip_path + "\\" + zip_file)
file

BadZipFile: zipfiles that span multiple disks are not supported

False
